Note that if $\sum_i \frac{1}{a_i^2} = \frac{1}{2}$, then for any odd prime $p$, $\sum_j \frac{1}{a_{k_j}^2}$ must not have a prime factor $p$ in its denominator, where $\{a_{k_j}\} = \{a_i \mid p | a_i\}$. With this knowledge, we can start from large odd numbers, and for each $p$ find all $\{a_{k_j}\}$ combos that satisfy this property. Any $p$-multiple not appearing in any of these $p$-combos can be eliminated from all future search (with other $p$s).

Once we have found all $p$-combos for all odd prime $p$'s, note that a $p_1$-combo $C_1$ and a $p_2$-combo $C_2$ are compatible iff there's no $n$ in range s.t. $p_1 | n$ and $n \in C_2 \setminus C_1$, or $p_2 | n$ and $n \in C_1 \setminus C_2$. Therefore we can do a breadth first search to find all "fat combos" combining compatible $p$-combos for all $p$'s.

Having completed the two steps above, with a narrow set of "fat combos" on our hands, and only powers of 2 to add to the equation, we can brute force however we want. (Do note that $\frac{1}{(2^k)^2} > \sum_{i=k+1}^\infty \frac{1}{(2^i)^2}$, so we definitely don't need to try all 2-combos).

Note that the first step of finding $p$-combos could probably take some hints from step two and be greatly optimized, but given the range at question, only finding 3-combos is remotely time-consuming, so I didn't bother to optimize further.

In [1]:
#!/usr/bin/env python3

import math
from collections import deque
from fractions import Fraction

import primesieve


LIMIT = 80
USABLE = [True] * (LIMIT + 1)


def find_prime_factor_combos(p):
    print(f"considering prime factor {p}...")
    potential = [n for n in range(p, LIMIT + 1, p) if USABLE[n]]
    k = len(potential)
    combos = []
    current_sum = Fraction(0)
    arr = [0] * k
    for _ in range(2 ** k - 1):
        i = 0
        while arr[i] == 1:
            arr[i] = 0
            current_sum -= Fraction(1, potential[i] ** 2)
            i += 1
        arr[i] = 1
        current_sum += Fraction(1, potential[i] ** 2)
        if current_sum.denominator % p != 0 and current_sum <= 1 / 2:
            combo = tuple(potential[i] for i in range(k) if arr[i])
            combos.append(combo)
    combos = sorted(combos)
    rejected = set(potential).difference(*combos)
    for n in rejected:
        USABLE[n] = False
    if combos:
        print(f"found {len(combos)} combos")
        # for combo in combos:
        #     print(combo)
    return [set(combo) for combo in combos]


def find_fat_combos(single_factor_combos):
    print("checking compatibility...")
    # A FIFO queue for BFS. Each element is of the form
    #
    #   (depth, combo, forbidden): (int, set, set)
    #
    # where depth is the next index in single_factor_combos to look,
    # combo is the current fat combo, and forbidden is the known set of
    # numbers forbidden by the combo.
    queue = deque()
    queue.append((0, set(), set()))
    maxdepth = len(single_factor_combos)
    while queue and queue[0][0] < maxdepth:
        depth, combo, forbidden = queue.popleft()
        p, p_combos = single_factor_combos[depth]
        existing_p_multiples = set(n for n in combo if n % p == 0)
        for p_combo in p_combos:
            if existing_p_multiples <= p_combo and not (p_combo & forbidden):
                new_combo = combo.copy()
                new_forbidden = forbidden.copy()
                new_combo |= p_combo
                new_forbidden |= set(range(p, LIMIT + 1, p)) - p_combo
                queue.append((depth + 1, new_combo, new_forbidden))
    fat_combos = [combo for _, combo, _ in queue]
    print(f"found {len(fat_combos)} fat combos")
    # for fat_combo in sorted(tuple(sorted(combo)) for combo in fat_combos):
    #     print(fat_combo)
    return fat_combos


def find_result_combos(fat_combos):
    print("searching for results...")
    choices = []
    n = 2
    while n <= LIMIT:
        choices.append((n, Fraction(1, n * n)))
        n *= 2
    results = []
    for combo in fat_combos:
        inverse_square_sum = sum(Fraction(1, n * n) for n in combo)
        remaining = Fraction(1, 2) - inverse_square_sum
        for n, contrib in choices:
            if remaining == contrib:
                combo.add(n)
                results.append(combo)
                break
            if remaining < contrib:
                continue
            combo.add(n)
            remaining -= contrib
    results = sorted(tuple(sorted(combo)) for combo in results)
    for result in results:
        print(result)
    print(f"found {len(results)} results")


def main():
    primes = primesieve.primes(3, LIMIT)
    single_factor_combos = []
    for p in reversed(primes):
        combos = find_prime_factor_combos(p)
        if combos:
            single_factor_combos.insert(0, (p, [set()] + combos))
    fat_combos = find_fat_combos(single_factor_combos)
    find_result_combos(fat_combos)


if __name__ == "__main__":
    main()


onsidering prime factor 41...
considering prime factor 37...
considering prime factor 31...
considering prime factor 29...
considering prime factor 23...
considering prime factor 19...
considering prime factor 17...
considering prime factor 13...
found 1 combos
considering prime factor 11...
considering prime factor 7...
found 19 combos
considering prime factor 5...
found 95 combos
considering prime factor 3...
found 1591 combos
checking compatibility...
found 6041 fat combos
searching for results...
(2, 3, 4, 5, 6, 21, 24, 30, 35, 36, 40, 45, 56, 60, 72)
(2, 3, 4, 5, 7, 9, 28, 35, 36, 45, 60)
(2, 3, 4, 5, 7, 10, 20, 28, 30, 35, 60)
(2, 3, 4, 5, 7, 10, 24, 28, 36, 40, 42, 45, 56, 72)
(2, 3, 4, 5, 7, 12, 13, 28, 35, 39, 52)
(2, 3, 4, 5, 7, 12, 15, 18, 36, 60, 63, 70)
(2, 3, 4, 5, 7, 12, 15, 20, 28, 35)
(2, 3, 4, 5, 7, 12, 15, 21, 28, 42, 60, 70)
(2, 3, 4, 5, 7, 12, 18, 21, 28, 35, 36, 42, 63)
(2, 3, 4, 5, 7, 12, 18, 21, 28, 35, 42, 45, 60, 63)
(2, 3, 4, 5, 7, 13, 15, 18, 36, 39, 52, 60,